In [ ]:
import sys
sys.path.append('../../')
import Config
import pickle
import rospy
rospy.init_node('phys_exec')

In [ ]:
# detect_plank_location = rospy.ServiceProxy('detect_plank_location', DetectPlank)
num_joints = 7
with open(Config.DOMAIN_DIR +'ros_traj.p', 'r') as f:
    trajectories = pickle.load(f)
    f.close()

## YUMI


In [ ]:
from yumipy import *
import time
import numpy as np
from plank_location_detection.srv import DetectPlank
import rospy
import importlib
from src.DataStructures.RefinedPolicy import RefinedPolicy
from src.DataStructures.RefinedPolicyNode import RefinedPolicyNode
from src.DataStructures.RefinedPolicyEdge import RefinedPolicyEdge

In [ ]:
def execute_gripper_close(yumi_robot,grip_force=10):
    return yumi_robot.left.close_gripper(force=grip_force, width=0., no_wait=False, wait_for_res=True)

def execute_gripper_open(yumi_robot):
    return  yumi_robot.left.open_gripper(no_wait=False, wait_for_res=True)

def go_to_state(yumi_robot,state):
    return  yumi_robot.left.goto_state(state, wait_for_res=True)
    # print(res)

def execute_manip(yumi_robot,trajectory):
    res = ''
    for i in trajectory.points:
        reordered_joints_in_radians = np.asarray(i.positions)[[0,1,3,4,5,6,2]] # TODO fix in openrave
        reordered_joints_in_degrees = np.degrees(reordered_joints_in_radians)
        state = YuMiState(list(reordered_joints_in_degrees))
        res = go_to_state(yumi_robot,state)
    return res


In [ ]:
yumi_robot = YuMiRobot()
vel = YuMiRobot.get_v(50)
yumi_robot.left.set_speed(vel)
yumi_robot.left._motion_timeout = 100
yumi_robot.left._comm_timeout = 100
yumi_robot.left._process_timeout = 100

In [ ]:
# Set Yumi to init State:
state = YuMiState([0.0, -129.99, 30.01, 0.0, 40.05, -0.01, 134.99])
ret = go_to_state(yumi_robot,state)

## FETCH

In [ ]:
sys.path.append(Config.MISC_DIR+"real_world_exec/")
from fetch.fetch_control import Gripper, Head
from control_msgs.msg import FollowJointTrajectoryGoal, FollowJointTrajectoryAction
import rospy
import actionlib
import rospy
from openravepy import *

In [ ]:
gripper = Gripper()

In [ ]:
def execute_manip_traj_on_ros(ros_manip_traj):
    client = actionlib.SimpleActionClient('/arm_with_torso_controller/follow_joint_trajectory',
                                          FollowJointTrajectoryAction)
    client.wait_for_server()
    id_gen = actionlib.GoalIDGenerator()
    jtg = FollowJointTrajectoryGoal()
    jtg.trajectory = ros_manip_traj
    print("sending goal..")
    client.send_goal(jtg)
    client.wait_for_result()
    print("done")   

In [ ]:
def physical_traj_exec(t_type, t_val, robot):
    global gripper
    if robot == 'yumi':
        return
        if t_type == 'GripperOpenTrajectory':
            ret = execute_gripper_open(yumi_ryumi_robotobot)
        elif t_type == 'GripperCloseTrajectory':
            ret = execute_gripper_close()
        elif t_type == 'ManipTrajectory':
            ret = execute_manip(yumi_robot,t_val)
        elif t_type == 'HandChange':
            ret = execute_hand_change() # TODO: implement this
    else:
        if t_type == 'GripperOpenTrajectory':
            gripper.open()
        elif t_type == 'GripperCloseTrajectory':
            gripper.close()
        elif t_type == 'ManipTrajectory':
            execute_manip_traj_on_ros(t_val)

## Run trajectories

In [ ]:
def run_trajectory(policy_tree):
    i = 0
    # This function executes the trajectory on openrave and also creates ros trajectories
    root = policy_tree.get_root()
    q = [root]
    while len(q) > 0:
        node = q.pop()
        children = node.get_children()
        if len(children) > 0:
            selected_child = children[0]
            edge = policy_tree.get_edge(node, selected_child)
            if edge.has_mp:
                ll_plan = edge.ll_plan
                exec_sequence = edge.exec_seq
                robot_name = edge.generated_values['robot']
                print(robot_name)
                for arg in exec_sequence:
                    # Execute physical Trajectory here
                    wait = raw_input("Execute Traj {} {} for {} (y/n)".format(str(i), str(ll_plan[arg]['type']), robot_name))
                    if wait == 'n':
                        break
                    print("Executing Traj {} - {}".format(str(i), str(ll_plan[arg]['type'])))
                    val = physical_traj_exec(ll_plan[arg]['type'],ll_plan[arg]['value'], robot_name)
                    i+=1
            q.append(selected_child)
        else:
            break

In [ ]:
run_trajectory(trajectories)